# TFM: Predicción de la demanda energética - Parte 1: Carga de datos
#### Lara Palacios
## 0. Importación de librerías

In [13]:
import pandas as pd
import json
from datetime import datetime

## 1. Carga de datos

### 1.1. Archivo cupsInfo

#### 1.1.1 Carga desde ficheros

In [14]:
file1 = "zdatos_20230109/cupsInfo.json"

In [15]:
with open(file1) as f:
    data1 = json.load(f)

In [16]:
def flatten_cupsInfo(json_obj):
    flattened_data = []

    for cod_cliente, cliente_data in json_obj.items():
        #if cod_cliente in clientes: #para las pruebas
        cod_simel = cliente_data.get('cod_simel', None)
        tarifa = cliente_data.get('tarifa', None)
        cpostal = cliente_data.get('cpostal', None)
        municipio = cliente_data.get('municipio', None)
        provincia = cliente_data.get('provincia', None)
        comAutonoma = cliente_data.get('comAutonoma', None)
        autoconsumo = cliente_data.get('autoconsumo', None)
        potencia_generacion_kW = cliente_data.get('potencia_generacion_kW', None)
        esquema_medida = cliente_data.get('esquema_medida', None)
        subgrupo = cliente_data.get('subgrupo', None)
        alta_autoconsumo = cliente_data.get('alta_autoconsumo', None)
        baja_autoconsumo = cliente_data.get('baja_autoconsumo', None)
        potencia_contratada_kW = cliente_data.get('potencia_contratada_kW', {})
        p1 = potencia_contratada_kW.get('p1', None)
        p2 = potencia_contratada_kW.get('p2', None)
        p3 = potencia_contratada_kW.get('p3', None)
        p4 = potencia_contratada_kW.get('p4', None)
        p5 = potencia_contratada_kW.get('p5', None)
        p6 = potencia_contratada_kW.get('p6', None)

        flattened_data.append({
            'PK_CLIENTE': int(cod_cliente),
            'TARIFA': str(tarifa),
            'MUNICIPIO': str(municipio),
            'PROVINCIA': str(provincia),
            'KW_CONTRATADA_P1': float(p1),
            'KW_CONTRATADA_P2': float(p2),
            'KW_CONTRATADA_P3': float(p3),
            'KW_CONTRATADA_P4': float(p4),
            'KW_CONTRATADA_P5': float(p5),
            'KW_CONTRATADA_P6': float(p6),
            'AUTOCONSUMO': int(autoconsumo),
            'POTENCIA_GENERACION_KW': float(potencia_generacion_kW),
            'ESQUEMA_MEDIDA': str(esquema_medida),
            'SUBGRUPO': str(subgrupo),
            'ALTA_AUTOCONSUMO': datetime.strptime(alta_autoconsumo, '%Y-%m-%d'),
            'BAJA_AUTOCONSUMO': datetime.strptime(baja_autoconsumo, '%Y-%m-%d')
            })

    return flattened_data

In [17]:
datos1 = data1.get("datos", {})
flattened_data1 = flatten_cupsInfo(datos1)
df1 = pd.DataFrame(flattened_data1)

In [18]:
df1.shape

(11786, 16)

In [21]:
df1.isnull().sum()

PK_CLIENTE                0
TARIFA                    0
MUNICIPIO                 0
PROVINCIA                 0
KW_CONTRATADA_P1          0
KW_CONTRATADA_P2          0
KW_CONTRATADA_P3          0
KW_CONTRATADA_P4          0
KW_CONTRATADA_P5          0
KW_CONTRATADA_P6          0
AUTOCONSUMO               0
POTENCIA_GENERACION_KW    0
ESQUEMA_MEDIDA            0
SUBGRUPO                  0
ALTA_AUTOCONSUMO          0
BAJA_AUTOCONSUMO          0
dtype: int64

In [20]:
df1.to_csv('cupsInfo.csv', index = False)

#### 1.1.2 Carga desde .csv

In [24]:
df1 = pd.read_csv('cupsInfo.csv') 
df1.shape

(11786, 16)

### 1.2. Archivos cups

#### 1.2.1 Carga desde ficheros

In [4]:
import os
cwd = os.getcwd()
dir2 = cwd + "\\datos_20230109\\cups"
files = [dir2 + "\\" + file for file in os.listdir(dir2)]
files[0]

'C:\\Users\\larap\\OneDrive - Universidad de La Rioja\\TFM - Lara\\datos_20230109\\cups\\1.json'

In [5]:
len(files)

11786

In [6]:
def flatten_cups(json_obj):
    flattened_data = []
    
    for cod_cliente, cliente_data in json_obj.items():
        
            for fecha, fecha_data in cliente_data.items():
                for hora, hora_data in fecha_data.items():
                    entry = hora_data.get('entrada', None)
                    exit = hora_data.get('salida', None)

                    flattened_data.append({
                        'PK_CLIENTE': int(cod_cliente),
                        'PK_ANYOMESDIA': datetime.strptime(fecha, '%Y-%m-%d'),
                        'PK_HORA': int((int(hora)/100)-1),
                        'ENTRADA': entry,
                        'SALIDA': exit
                    })

    return flattened_data

In [7]:
df2 = pd.DataFrame()
i=0

for file2 in files:
    # Lee el archivo JSON
    with open(file2) as f:
        data2 = json.load(f)

    datos2 = data2.get("datos", {})

    # Aplana el JSON
    flattened_data2 = flatten_cups(datos2)
    df_concat2 = pd.DataFrame(flattened_data2)

    #concatenamos todos los dataframes
    df2 = pd.concat([df2, df_concat2], axis=0)
    
    i = i+1
    if i%500 == 0:
        print(f"{str(i)} archivos leídos.")

500 archivos leídos.
1000 archivos leídos.
1500 archivos leídos.
2000 archivos leídos.
2500 archivos leídos.
3000 archivos leídos.
3500 archivos leídos.
4000 archivos leídos.
4500 archivos leídos.
5000 archivos leídos.
5500 archivos leídos.
6000 archivos leídos.
6500 archivos leídos.
7000 archivos leídos.
7500 archivos leídos.
8000 archivos leídos.
8500 archivos leídos.
9000 archivos leídos.
9500 archivos leídos.
10000 archivos leídos.
10500 archivos leídos.
11000 archivos leídos.
11500 archivos leídos.


In [12]:
df2.isna().sum()

PK_CLIENTE             0
PK_ANYOMESDIA          0
PK_HORA                0
ENTRADA          1612265
SALIDA           1612288
dtype: int64

In [14]:
df2['PK_ANYOMESDIA'].describe()

C:\Users\larap\AppData\Local\Temp\ipykernel_22976\1366616956.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_2['PK_ANYOMESDIA'].describe()


count                17396856
unique                     62
top       2023-01-03 00:00:00
freq                   282816
first     2022-11-09 00:00:00
last      2023-01-09 00:00:00
Name: PK_ANYOMESDIA, dtype: object

In [15]:
df2.to_parquet('cups.parquet')

#### 1.2.2 Carga desde .parquet

In [31]:
df2 = pd.read_parquet('cups.parquet')
df2.shape

(17396856, 5)

In [32]:
df2 = df2.dropna().reset_index(drop = True)
df2.shape

(15784568, 5)

### 1.3. Archivos tiempo

#### 1.3.1 Carga desde ficheros

In [22]:
import os
cwd = os.getcwd()
dir3 = cwd + "\\datos_20230109\\tiempo"
clientes = [file for file in os.listdir(dir3)]
len(clientes)

11786

In [23]:
df_concatenado = pd.read_parquet('tiempo.parquet') 
df_concatenado.shape

(15397206, 16)

In [24]:
ya_procesado = df_concatenado['PK_CLIENTE'].unique().tolist()
len(ya_procesado)

10185

In [25]:
clientes_part2 = [cliente for cliente in clientes if int(cliente) not in ya_procesado]
clientes_part2

['1000',
 '10003',
 '1001',
 '10016',
 '1004',
 '10043',
 '10044',
 '10052',
 '1006',
 '1007',
 '10070',
 '10076',
 '10092',
 '1010',
 '10119',
 '10133',
 '10158',
 '10176',
 '1018',
 '10216',
 '1024',
 '10240',
 '10241',
 '10242',
 '10244',
 '1025',
 '1026',
 '10266',
 '10278',
 '10297',
 '1030',
 '10319',
 '10324',
 '10327',
 '10330',
 '10340',
 '10374',
 '10375',
 '10376',
 '10377',
 '10378',
 '1038',
 '1039',
 '1043',
 '1044',
 '1051',
 '1052',
 '1054',
 '1062',
 '1066',
 '10730',
 '1100',
 '1107',
 '11514',
 '11548',
 '11549',
 '11550',
 '11551',
 '11552',
 '11553',
 '11554',
 '11555',
 '11556',
 '11557',
 '11558',
 '11559',
 '11560',
 '11561',
 '11562',
 '11564',
 '11565',
 '11566',
 '11567',
 '11568',
 '11569',
 '11570',
 '11613',
 '11614',
 '11615',
 '11616',
 '11617',
 '11618',
 '11619',
 '11620',
 '11621',
 '11622',
 '11623',
 '11624',
 '11625',
 '11626',
 '11627',
 '11628',
 '11629',
 '11630',
 '11631',
 '11632',
 '11633',
 '11634',
 '11635',
 '11636',
 '11637',
 '11638',
 '

In [26]:
def flatten_tiempo(json_obj):
    flattened_data = []

    municipio = json_obj['municipio']
    fecha = datetime.strptime(json_obj['fecha'], '%Y-%m-%d')
    orto = datetime.strptime(json_obj['orto'], '%Y-%m-%d %H:%M:%S')
    ocaso = datetime.strptime(json_obj['ocaso'], '%Y-%m-%d %H:%M:%S')

    for hora, hora_data in json_obj.items():
        if hora.isdigit():
            nieve = hora_data.get('nieve', None)
            estado_cielo = hora_data.get('estadoCielo', None)
            descripcion = hora_data.get('descripcion', None)
            sens_termica = hora_data.get('sensTermica', None)
            temperatura = hora_data.get('temperatura', None)
            precipitacion = hora_data.get('precipitacion', None)
            humedad_relativa = hora_data.get('humedadRelativa', None)
            viento_direccion = hora_data.get('vientoDireccion', None)
            viento_velocidad = hora_data.get('vientoVelocidad', None)
            viento_and_racha_max = hora_data.get('vientoAndRachaMax', None)

            flattened_data.append({
                'PK_ANYOMESDIA': fecha,
                'PK_HORA': int(hora),
                'MUNICIPIO': str(municipio),
                'ORTO' : str(orto)[11:16],
                'OCASO': str(ocaso)[11:16],
                'NIEVE': nieve,
                'ESTADO_CIELO': estado_cielo,
                'DESCRIPCION': descripcion,
                'SENS_TERMICA': int(sens_termica),
                'TEMPERATURA': int(temperatura),
                'PRECIPITACION': precipitacion,
                'HUMEDAD_RELATIVA': humedad_relativa,
                'VIENTO_DIRECCION': viento_direccion,
                'VIENTO_VELOCIDAD': viento_velocidad,
                'VIENTO_AND_RACHA_MAX': viento_and_racha_max
            })

    return flattened_data

In [27]:
df3 = pd.DataFrame()
i = 0
j = 0

for cliente3 in clientes_part2:
    dir3_2 = dir3 + "\\" + cliente3
    files3 = [dir3_2 + "\\" + file for file in os.listdir(dir3_2)]
    print('El cliente ' + str(cliente3) + ' tiene ' + str(len(files3)) + ' archivos.')
    for file3 in files3:
        # Lee el archivo JSON
        with open(file3) as f:
            data3 = json.load(f)

        datos3 = data3.get("datos", {})
        
        try:
            # Aplana el JSON
            flattened_data3 = flatten_tiempo(datos3)
            df3_aux = pd.DataFrame(flattened_data3)
            df3_aux.insert(0, 'PK_CLIENTE', int(cliente3))

            #concatenamos todos los dataframes
            df3 = pd.concat([df3, df3_aux], axis=0)
        except TypeError as e:
            print('Ha fallado el fichero ' + str(file3) + ' del cliente ' + str(cliente3) + ' con el TypeError ' + str(e))   
        
    i = i +1
    if i%500 == 0:
        print(f"{str(i)} clientes leídos.")

El cliente 1000 tiene 0 archivos.
El cliente 10003 tiene 0 archivos.
El cliente 1001 tiene 0 archivos.
El cliente 10016 tiene 0 archivos.
El cliente 1004 tiene 0 archivos.
El cliente 10043 tiene 0 archivos.
El cliente 10044 tiene 0 archivos.
El cliente 10052 tiene 0 archivos.
El cliente 1006 tiene 0 archivos.
El cliente 1007 tiene 0 archivos.
El cliente 10070 tiene 0 archivos.
El cliente 10076 tiene 0 archivos.
El cliente 10092 tiene 0 archivos.
El cliente 1010 tiene 0 archivos.
El cliente 10119 tiene 0 archivos.
El cliente 10133 tiene 0 archivos.
El cliente 10158 tiene 0 archivos.
El cliente 10176 tiene 0 archivos.
El cliente 1018 tiene 0 archivos.
El cliente 10216 tiene 0 archivos.
El cliente 1024 tiene 0 archivos.
El cliente 10240 tiene 0 archivos.
El cliente 10241 tiene 0 archivos.
El cliente 10242 tiene 0 archivos.
El cliente 10244 tiene 0 archivos.
El cliente 1025 tiene 0 archivos.
El cliente 1026 tiene 0 archivos.
El cliente 10266 tiene 0 archivos.
El cliente 10278 tiene 0 archi

El cliente 3010 tiene 0 archivos.
El cliente 3015 tiene 0 archivos.
El cliente 302 tiene 0 archivos.
El cliente 3243 tiene 0 archivos.
El cliente 34 tiene 0 archivos.
El cliente 345 tiene 0 archivos.
El cliente 346 tiene 0 archivos.
El cliente 3630 tiene 0 archivos.
El cliente 3640 tiene 0 archivos.
El cliente 367 tiene 0 archivos.
El cliente 368 tiene 0 archivos.
El cliente 369 tiene 0 archivos.
El cliente 386 tiene 0 archivos.
El cliente 3866 tiene 0 archivos.
El cliente 396 tiene 0 archivos.
El cliente 401 tiene 0 archivos.
El cliente 402 tiene 0 archivos.
El cliente 406 tiene 0 archivos.
El cliente 409 tiene 0 archivos.
El cliente 411 tiene 0 archivos.
El cliente 413 tiene 0 archivos.
El cliente 4159 tiene 0 archivos.
El cliente 418 tiene 0 archivos.
El cliente 419 tiene 0 archivos.
El cliente 4271 tiene 0 archivos.
El cliente 4302 tiene 0 archivos.
El cliente 433 tiene 0 archivos.
El cliente 434 tiene 0 archivos.
El cliente 436 tiene 0 archivos.
El cliente 437 tiene 0 archivos.
El

El cliente 9088 tiene 64 archivos.
El cliente 9089 tiene 64 archivos.
El cliente 909 tiene 64 archivos.
El cliente 9090 tiene 64 archivos.
El cliente 9091 tiene 64 archivos.
El cliente 9092 tiene 64 archivos.
El cliente 9093 tiene 64 archivos.
El cliente 9094 tiene 64 archivos.
El cliente 9095 tiene 64 archivos.
El cliente 9096 tiene 64 archivos.
El cliente 9097 tiene 64 archivos.
El cliente 9098 tiene 64 archivos.
El cliente 9099 tiene 64 archivos.
El cliente 91 tiene 63 archivos.
El cliente 910 tiene 64 archivos.
El cliente 9100 tiene 64 archivos.
El cliente 9101 tiene 64 archivos.
El cliente 9102 tiene 64 archivos.
El cliente 9103 tiene 64 archivos.
El cliente 9104 tiene 64 archivos.
El cliente 9105 tiene 64 archivos.
El cliente 9106 tiene 64 archivos.
El cliente 9107 tiene 64 archivos.
El cliente 9108 tiene 64 archivos.
El cliente 9109 tiene 64 archivos.
El cliente 911 tiene 64 archivos.
El cliente 9110 tiene 64 archivos.
El cliente 9111 tiene 64 archivos.
El cliente 9112 tiene 64 

El cliente 93 tiene 63 archivos.
El cliente 930 tiene 64 archivos.
El cliente 9300 tiene 64 archivos.
El cliente 9301 tiene 64 archivos.
El cliente 9302 tiene 64 archivos.
El cliente 9303 tiene 64 archivos.
El cliente 9304 tiene 64 archivos.
El cliente 9305 tiene 64 archivos.
El cliente 9306 tiene 64 archivos.
El cliente 9307 tiene 64 archivos.
El cliente 9308 tiene 64 archivos.
El cliente 9309 tiene 64 archivos.
El cliente 931 tiene 64 archivos.
El cliente 9310 tiene 64 archivos.
El cliente 9311 tiene 64 archivos.
El cliente 9312 tiene 64 archivos.
El cliente 9313 tiene 64 archivos.
El cliente 9314 tiene 64 archivos.
El cliente 9315 tiene 64 archivos.
El cliente 9316 tiene 64 archivos.
El cliente 9317 tiene 64 archivos.
El cliente 9318 tiene 64 archivos.
El cliente 9319 tiene 64 archivos.
El cliente 932 tiene 0 archivos.
El cliente 9320 tiene 64 archivos.
El cliente 9321 tiene 64 archivos.
El cliente 9322 tiene 64 archivos.
El cliente 9323 tiene 64 archivos.
El cliente 9324 tiene 64 a

El cliente 9510 tiene 64 archivos.
El cliente 9511 tiene 64 archivos.
El cliente 9512 tiene 64 archivos.
El cliente 9513 tiene 64 archivos.
El cliente 9514 tiene 64 archivos.
El cliente 9515 tiene 64 archivos.
El cliente 9516 tiene 64 archivos.
El cliente 9517 tiene 64 archivos.
El cliente 9518 tiene 64 archivos.
El cliente 9519 tiene 64 archivos.
El cliente 952 tiene 64 archivos.
El cliente 9520 tiene 64 archivos.
El cliente 9521 tiene 64 archivos.
El cliente 9522 tiene 64 archivos.
El cliente 9523 tiene 64 archivos.
El cliente 9524 tiene 64 archivos.
El cliente 9525 tiene 64 archivos.
El cliente 9526 tiene 64 archivos.
El cliente 9527 tiene 64 archivos.
El cliente 9528 tiene 64 archivos.
El cliente 9529 tiene 64 archivos.
El cliente 953 tiene 0 archivos.
El cliente 9530 tiene 64 archivos.
El cliente 9531 tiene 64 archivos.
El cliente 9532 tiene 64 archivos.
El cliente 9533 tiene 64 archivos.
El cliente 9534 tiene 63 archivos.
El cliente 9535 tiene 64 archivos.
El cliente 9536 tiene 6

El cliente 9722 tiene 64 archivos.
El cliente 9723 tiene 64 archivos.
El cliente 9724 tiene 64 archivos.
El cliente 9725 tiene 0 archivos.
El cliente 9726 tiene 64 archivos.
El cliente 9727 tiene 64 archivos.
El cliente 9728 tiene 64 archivos.
El cliente 9729 tiene 0 archivos.
El cliente 973 tiene 64 archivos.
El cliente 9730 tiene 64 archivos.
El cliente 9731 tiene 64 archivos.
El cliente 9732 tiene 64 archivos.
El cliente 9733 tiene 64 archivos.
El cliente 9734 tiene 64 archivos.
El cliente 9735 tiene 64 archivos.
El cliente 9736 tiene 64 archivos.
El cliente 9737 tiene 64 archivos.
El cliente 9738 tiene 64 archivos.
El cliente 9739 tiene 64 archivos.
El cliente 974 tiene 64 archivos.
El cliente 9740 tiene 64 archivos.
El cliente 9741 tiene 64 archivos.
El cliente 9742 tiene 64 archivos.
El cliente 9743 tiene 64 archivos.
El cliente 9744 tiene 64 archivos.
El cliente 9745 tiene 0 archivos.
El cliente 9746 tiene 64 archivos.
El cliente 9747 tiene 64 archivos.
El cliente 9748 tiene 64 

El cliente 9935 tiene 64 archivos.
El cliente 9936 tiene 64 archivos.
El cliente 9937 tiene 64 archivos.
El cliente 9938 tiene 64 archivos.
El cliente 9939 tiene 64 archivos.
El cliente 994 tiene 64 archivos.
El cliente 9940 tiene 64 archivos.
El cliente 9941 tiene 64 archivos.
El cliente 9942 tiene 64 archivos.
El cliente 9943 tiene 64 archivos.
El cliente 9944 tiene 64 archivos.
El cliente 9945 tiene 64 archivos.
El cliente 9946 tiene 64 archivos.
El cliente 9947 tiene 0 archivos.
El cliente 9948 tiene 64 archivos.
El cliente 9949 tiene 64 archivos.
El cliente 995 tiene 0 archivos.
El cliente 9950 tiene 64 archivos.
El cliente 9951 tiene 64 archivos.
El cliente 9952 tiene 64 archivos.
El cliente 9953 tiene 64 archivos.
El cliente 9954 tiene 64 archivos.
El cliente 9955 tiene 64 archivos.
El cliente 9956 tiene 64 archivos.
El cliente 9957 tiene 64 archivos.
El cliente 9958 tiene 64 archivos.
El cliente 9959 tiene 0 archivos.
El cliente 996 tiene 0 archivos.
El cliente 9960 tiene 64 ar

In [28]:
df3

,PK_CLIENTE,PK_ANYOMESDIA,PK_HORA,MUNICIPIO,ORTO,OCASO,NIEVE,ESTADO_CIELO,DESCRIPCION,SENS_TERMICA,TEMPERATURA,PRECIPITACION,HUMEDAD_RELATIVA,VIENTO_DIRECCION,VIENTO_VELOCIDAD,VIENTO_AND_RACHA_MAX
0,8947,2022-11-09,0,08295,07:33,17:38,0,17n,Nubes altas,13,13,0,87,O,7,9
1,8947,2022-11-09,1,08295,07:33,17:38,0,12n,Poco nuboso,13,13,0,89,O,6,12
2,8947,2022-11-09,2,08295,07:33,17:38,0,17n,Nubes altas,14,14,0,92,O,3,4
3,8947,2022-11-09,3,08295,07:33,17:38,0,17n,Nubes altas,15,15,0,93,NO,3,9
4,8947,2022-11-09,4,08295,07:33,17:38,0,17n,Nubes altas,13,13,0,95,NO,6,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,9999,2023-01-10,20,07022,08:07,17:42,0,11n,Despejado,10,10,0,75,SO,4,6
21,9999,2023-01-10,21,07022,08:07,17:42,0,11n,Despejado,7,7,0,84,SO,5,6
22,9999,2023-01-10,22,07022,08:07,17:42,0,11n,Despejado,8,8,0,90,SO,5,7
23,9999,2023-01-10,23,07022,08:07,17:42,0,11n,Despejado,9,9,0,91,O,5,7


In [9]:
df3[df3['PK_CLIENTE']== 8947]

,PK_CLIENTE,PK_ANYOMESDIA,PK_HORA,MUNICIPIO,ORTO,OCASO,NIEVE,ESTADO_CIELO,DESCRIPCION,SENS_TERMICA,TEMPERATURA,PRECIPITACION,HUMEDAD_RELATIVA,VIENTO_DIRECCION,VIENTO_VELOCIDAD,VIENTO_AND_RACHA_MAX
0,8947,2022-11-09,0,08295,07:33,17:38,0,17n,Nubes altas,13,13,0,87,O,7,9
1,8947,2022-11-09,1,08295,07:33,17:38,0,12n,Poco nuboso,13,13,0,89,O,6,12
2,8947,2022-11-09,2,08295,07:33,17:38,0,17n,Nubes altas,14,14,0,92,O,3,4
3,8947,2022-11-09,3,08295,07:33,17:38,0,17n,Nubes altas,15,15,0,93,NO,3,9
4,8947,2022-11-09,4,08295,07:33,17:38,0,17n,Nubes altas,13,13,0,95,NO,6,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,8947,2023-01-03,19,08295,08:18,17:35,0,82n,Bruma,10,10,0,95,NO,5,6
20,8947,2023-01-03,20,08295,08:18,17:35,0,81n,Niebla,10,10,0,98,O,3,9
21,8947,2023-01-03,21,08295,08:18,17:35,0,12n,Poco nuboso,9,9,0,96,NO,5,11
22,8947,2023-01-03,22,08295,08:18,17:35,0,11n,Despejado,9,9,0,92,O,6,12


In [10]:
df3 = df3[:-1344]

In [29]:
df3.shape

(1687139, 16)

In [30]:
df_3 = df3

In [31]:
df_3.to_parquet('tiempo_parte_13.parquet')

#### 1.3.2 Carga desde .parquet

In [32]:
df_3 = pd.read_parquet('tiempo.parquet') 
df_3.shape

(15397206, 16)

In [33]:
df_4 = pd.read_parquet('tiempo_parte_13.parquet') 
df_4.shape

(1687139, 16)

In [34]:
df_concatenado = pd.concat([df_3, df_4])

In [35]:
df_concatenado.shape

(17084345, 16)

In [38]:
df_concatenado.drop_duplicates().shape

(17084345, 16)

In [36]:
df_concatenado.to_parquet('tiempo.parquet')